## обработка данных данных

In [ ]:
#linear algebra
import numpy as np
#data structures
import pandas as pd
#ml models
import scipy as sp
import sklearn
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.svm import SVR
#plots
import matplotlib.pyplot as plt
%matplotlib inline
#beautiful plots
import seaborn as sns
#linear regression

#set style for plots
sns.set_style('darkgrid')
#off the warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('train.tsv',sep="	")
hh_dt = pd.read_csv('hh_data.csv')
hh_dt = hh_dt.drop(columns='Unnamed: 0')

In [ ]:
data

,id,name,employer_name,experience_name,schedule_name,key_skills_name,accept_handicapped,accept_kids,unified_address_city,unified_address_state,...,raw_branded_description,lemmaized_wo_stopwords_raw_description,lemmaized_wo_stopwords_raw_branded_description,if_foreign_language,is_branded_description,name_clean,employment_name,employer_id,employer_industries,salary_mean_net
0,49242400,Инженер группы технической поддержки,DatsTeam,От 1 года до 3 лет,Сменный график,не указано,False,False,Москва,Москва,...,NaN,разрабатывать рекламный система оплата действи...,NaN,Не указано,заполнено,инженер группа технический поддержка,Полная занятость,4437201.0,не указано,80000.00
1,49496801,Специалист группы кадрового администрирования,Почта России,Нет опыта,Полный день,оформление трудовых книжек,False,False,Абакан,Республика Хакасия,...,АО «Почта России» входит в перечень системообр...,обязанность ведение кадровый делопроизводство ...,ао почта россия входить перечень системообразу...,Не указано,не заполнено,специалист группа кадровый администрирование,Полная занятость,4352.0,почтовая доставка,19989.12
2,44198403,Оператор 1С,Стройгарантсервис,От 1 года до 3 лет,Полный день,первичные документы,False,False,Москва,Москва,...,NaN,условие график работа 5/2 09.00 18.00;• оформл...,NaN,Не указано,заполнено,оператор 1с,Полная занятость,1794943.0,лифты,45000.00
3,47707600,Инженер ПТО,Коммунальник,От 3 до 6 лет,Полный день,autocad,False,False,NaN,Краснодарский край,...,NaN,строительный инженерный компания коммунальник ...,NaN,Не указано,заполнено,инженер пто,Полная занятость,1881241.0,не указано,40000.00
4,44610002,Водитель в войсковую часть (пункт отбора на во...,Пункт отбора на военную службу по контракту по...,Нет опыта,Полный день,водительское удостоверение категории bc,False,False,NaN,Калужская область,...,NaN,обязанность знать устройство правило эксплуата...,NaN,Не указано,заполнено,водитель войсковой часть пункт отбор военный с...,Полная занятость,1586217.0,не указано,37500.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61309,49132201,Инженер ОТК,РГМ-Нефть-Газ-Сервис,От 1 года до 3 лет,Полный день,не указано,False,False,Воронеж,Воронежская область,...,NaN,обязанность контроль сварка вика контроль дета...,NaN,Не указано,заполнено,инженер отк,Полная занятость,1089764.0,не указано,30450.00
61310,41386400,Врач-эндокринолог (г. Геленджик),ГКУ КК ЦЗН города Геленджик,Нет опыта,Полный день,не указано,False,False,NaN,Калининградская область,...,NaN,обязанность выполнять перечень работа услуга д...,NaN,Не указано,заполнено,врач эндокринолог г. геленджик,Полная занятость,4481442.0,не указано,30450.00
61311,43351202,Швея-портной,Дом Быта,От 3 до 6 лет,Полный день,работа с клиентами,False,False,Москва,Москва,...,NaN,обязанность ремонт реставрация одежда перекрой...,NaN,Не указано,заполнено,швея портной,Полная занятость,4159812.0,не указано,60000.00
61312,44745603,Специалист помощи пользователям Яндекс.Учебника,Яндекс Крауд,Нет опыта,Удаленная работа,не указано,True,False,Пушкино,Московская область,...,NaN,день сервис яндекс пользоваться миллион челове...,NaN,Не указано,заполнено,специалист помощь пользователь яндекс учебник,Частичная занятость,9498112.0,не указано,13000.00


In [ ]:
hh_dt.columns

Index(['experience_name', 'schedule_name', 'unified_address_state',
       'professional_roles_name', 'salary_mean_net'],
      dtype='object')

In [ ]:
data.info()

In [ ]:
(data.lemmaized_wo_stopwords_raw_description
)

0        разрабатывать рекламный система оплата действи...
1        обязанность ведение кадровый делопроизводство ...
2        условие график работа 5/2 09.00 18.00;• оформл...
3        строительный инженерный компания коммунальник ...
4        обязанность знать устройство правило эксплуата...
                               ...                        
61309    обязанность контроль сварка вика контроль дета...
61310    обязанность выполнять перечень работа услуга д...
61311    обязанность ремонт реставрация одежда перекрой...
61312    день сервис яндекс пользоваться миллион челове...
61313    petrushka первый сеть ресторан быстрый питание...
Name: lemmaized_wo_stopwords_raw_description, Length: 61314, dtype: object

In [ ]:
city_to_area = {}
missing_cities = []


for i in hh_dt.unified_address_state:
    city_rows = data[data.unified_address_city == i].unified_address_state.to_list()
    if len(city_rows) == 0:
        missing_cities.append(i)
        continue
    city_to_area[i] = city_rows[0]



In [ ]:
hh_dt.drop(index=hh_dt[hh_dt.unified_address_state.isin(missing_cities)].index, inplace=True)

In [ ]:
hh_dt.area = hh_dt.unified_address_state.map(city_to_area)

In [ ]:
data.unified_address_country.unique()

array(['Россия'], dtype=object)

In [ ]:
dt = data[['experience_name', 'schedule_name', 'unified_address_state', 'professional_roles_name', 'salary_mean_net', 'lemmaized_wo_stopwords_raw_description']]

dt

,experience_name,schedule_name,unified_address_state,professional_roles_name,salary_mean_net,lemmaized_wo_stopwords_raw_description
0,От 1 года до 3 лет,Сменный график,Москва,специалист технической поддержки,80000.00,разрабатывать рекламный система оплата действи...
1,Нет опыта,Полный день,Республика Хакасия,специалист по кадрам,19989.12,обязанность ведение кадровый делопроизводство ...
2,От 1 года до 3 лет,Полный день,Москва,оператор базы данных,45000.00,условие график работа 5/2 09.00 18.00;• оформл...
3,От 3 до 6 лет,Полный день,Краснодарский край,инженер-проектировщик,40000.00,строительный инженерный компания коммунальник ...
4,Нет опыта,Полный день,Калужская область,водитель,37500.00,обязанность знать устройство правило эксплуата...
...,...,...,...,...,...,...
61309,От 1 года до 3 лет,Полный день,Воронежская область,технолог,30450.00,обязанность контроль сварка вика контроль дета...
61310,Нет опыта,Полный день,Калининградская область,врач,30450.00,обязанность выполнять перечень работа услуга д...
61311,От 3 до 6 лет,Полный день,Москва,закройщик,60000.00,обязанность ремонт реставрация одежда перекрой...
61312,Нет опыта,Удаленная работа,Московская область,другое,13000.00,день сервис яндекс пользоваться миллион челове...


experience_name

0 - нет опыта

1 - от 1 года до 3 лет

2 - от 3 до 6 лет

3 - более 6 лет

schedule_name

0 - сменный график

1 - полный день

2 - гибкий график

3 - вахтовый метод

4 - удаленная работа

accept_handicapped

0 - False

1 - True

accept_kids

0 - False

1 - True

unified_address_state

см states

индексы - номера

professional_roles_name

см states

индексы - номера


In [ ]:
hh_dt['schedule_name'].replace('Сменный график', 0, inplace=True)
hh_dt['schedule_name'].replace('Полный день', 1, inplace=True)
hh_dt['schedule_name'].replace('Гибкий график', 2, inplace=True)
hh_dt['schedule_name'].replace('Вахтовый метод', 3, inplace=True)
hh_dt['schedule_name'].replace('Удаленная работа', 4, inplace=True)

dt['schedule_name'].replace('Сменный график', 0, inplace=True)
dt['schedule_name'].replace('Полный день', 1, inplace=True)
dt['schedule_name'].replace('Гибкий график', 2, inplace=True)
dt['schedule_name'].replace('Вахтовый метод', 3, inplace=True)
dt['schedule_name'].replace('Удаленная работа', 4, inplace=True)

In [ ]:
hh_dt['experience_name'].replace('Нет опыта', 0, inplace=True)
hh_dt['experience_name'].replace('От 1 года до 3 лет', 1, inplace=True)
hh_dt['experience_name'].replace('От 3 до 6 лет', 2, inplace=True)
hh_dt['experience_name'].replace('Более 6 лет', 3, inplace=True)

dt['experience_name'].replace('Нет опыта', 0, inplace=True)
dt['experience_name'].replace('От 1 года до 3 лет', 1, inplace=True)
dt['experience_name'].replace('От 3 до 6 лет', 2, inplace=True)
dt['experience_name'].replace('Более 6 лет', 3, inplace=True)

In [ ]:
dt

,experience_name,schedule_name,unified_address_state,professional_roles_name,salary_mean_net,lemmaized_wo_stopwords_raw_description
0,1,0,Москва,специалист технической поддержки,80000.00,разрабатывать рекламный система оплата действи...
1,0,1,Республика Хакасия,специалист по кадрам,19989.12,обязанность ведение кадровый делопроизводство ...
2,1,1,Москва,оператор базы данных,45000.00,условие график работа 5/2 09.00 18.00;• оформл...
3,2,1,Краснодарский край,инженер-проектировщик,40000.00,строительный инженерный компания коммунальник ...
4,0,1,Калужская область,водитель,37500.00,обязанность знать устройство правило эксплуата...
...,...,...,...,...,...,...
61309,1,1,Воронежская область,технолог,30450.00,обязанность контроль сварка вика контроль дета...
61310,0,1,Калининградская область,врач,30450.00,обязанность выполнять перечень работа услуга д...
61311,2,1,Москва,закройщик,60000.00,обязанность ремонт реставрация одежда перекрой...
61312,0,4,Московская область,другое,13000.00,день сервис яндекс пользоваться миллион челове...


In [ ]:
states = []

for i in dt.unified_address_state:
  if i not in states and isinstance(i, int) is False:
    states.append(i)
    dt['unified_address_state'].replace(i, len(states) - 1, inplace=True)

states = []

for i in hh_dt.unified_address_state:
  if i not in states and isinstance(i, int) is False:
    states.append(i)
    hh_dt['unified_address_state'].replace(i, len(states) - 1, inplace=True)
    

In [ ]:
prof = []

for i in dt.professional_roles_name:
  if i not in prof and isinstance(i, int) is False:
    prof.append(i)
    dt['professional_roles_name'].replace(i, len(prof) - 1, inplace=True)

prof = []

for i in hh_dt.professional_roles_name:
  if i not in prof and isinstance(i, int) is False:
    prof.append(i)
    hh_dt['professional_roles_name'].replace(i, len(prof) - 1, inplace=True)

In [ ]:
print(prof)
print(len(prof))
dt

['менеджер по продажам, менеджер по работе с клиентами', 'курьер', 'упаковщик, комплектовщик', 'оператор call-центра, специалист контактного центра', 'продавец-консультант, продавец-кассир', 'официант, бармен, бариста', 'водитель', 'кредитный специалист', 'администратор', 'тестировщик', 'другое', 'делопроизводитель, архивариус', 'офис-менеджер', 'менеджер по маркетингу, интернет-маркетолог', 'швея, портной, закройщик', 'уборщица, уборщик', 'экономист', 'кладовщик', 'режиссер, сценарист', 'менеджер по закупкам', 'специалист по тендерам', 'агент по недвижимости', 'артист, актер, аниматор', 'оператор производственной линии', 'оператор пк, оператор базы данных', 'электромонтажник', 'журналист, корреспондент', 'генеральный директор, исполнительный директор (ceo)', 'менеджер по логистике, менеджер по вэд', 'администратор магазина, администратор торгового зала', 'специалист службы безопасности', 'прораб, мастер смр', 'мастер по ремонту оборудования, техники', 'руководитель отдела продаж', 'бу

,experience_name,schedule_name,unified_address_state,professional_roles_name,salary_mean_net,lemmaized_wo_stopwords_raw_description
0,1,0,0,0,80000.00,разрабатывать рекламный система оплата действи...
1,0,1,1,1,19989.12,обязанность ведение кадровый делопроизводство ...
2,1,1,0,2,45000.00,условие график работа 5/2 09.00 18.00;• оформл...
3,2,1,2,3,40000.00,строительный инженерный компания коммунальник ...
4,0,1,3,4,37500.00,обязанность знать устройство правило эксплуата...
...,...,...,...,...,...,...
61309,1,1,11,31,30450.00,обязанность контроль сварка вика контроль дета...
61310,0,1,62,20,30450.00,обязанность выполнять перечень работа услуга д...
61311,2,1,0,48,60000.00,обязанность ремонт реставрация одежда перекрой...
61312,0,4,27,7,13000.00,день сервис яндекс пользоваться миллион челове...


In [ ]:
roles_to_drop = [5]

dt.drop(dt[dt.professional_roles_name.isin(roles_to_drop)].index, inplace=True)
#удаляем строчки, проффесии которых встречаются до 50 раз

In [ ]:
ser = dt.unified_address_state.value_counts()

In [ ]:
states_to_drop = list(ser[ser < 50].index)

dt.drop(dt[dt.unified_address_state.isin(states_to_drop)].index, inplace=True)
#удаляем строчки, округов которые встречаются до 50 раз

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# corpus = dt.lemmaized_wo_stopwords_raw_description.to_list()

# vectorizer = TfidfVectorizer()
# vectors = vectorizer.fit_transform(corpus)

# vectors

<61066x77632 sparse matrix of type '<class 'numpy.float64'>'
	with 5115237 stored elements in Compressed Sparse Row format>

In [ ]:
# import numpy as np
# from sklearn.decomposition import SparsePCA, TruncatedSVD
# transformer = TruncatedSVD(n_components=200, random_state=0)
# transformer.fit(vectors)
# vectors_transformed = transformer.transform(vectors)
# vectors_transformed.shape
# np.mean(transformer.components_ == 0)

0.0

In [ ]:
# import numpy as np
# with open('vectors.npy', 'wb') as f:
#     np.save(f, vectors_transformed,allow_pickle=True)

In [ ]:
with open('vectors.npy', 'rb') as f:
    vectors_transformed = np.load(f,allow_pickle=True)

In [ ]:
dt

## Графички

In [ ]:
# график исходных данных
import seaborn as sns

sns.set(style="white")
corr = dt.drop(columns=['lemmaized_wo_stopwords_raw_description']).corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 11, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio

sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot = True, vmin=-1, vmax=1, title = "Корреляция исходных данных")
plt.show()

ValueError: could not convert string to float: 'разрабатывать рекламный система оплата действие высоконагруженный проект платформа агрегатор товар приложение мобильный приложение финтех продукт отличный команда возможность применение навык развитие новый скиллов система грейдирование пересмотр зп лояльность отличный бонус вид веранда вид москва река необходимый комфортный работа приглашать команда технический поддержка пользователь коллега офис сменный график цикл 2 день офис 2 выходной 2 ночь хоумофис 2 выходной должностной обязанность поддержка пользователь центральный офис филиал компания обеспечение бесперебойный работа офисный техника пк оргтехника заведение новый пользователь управление доступ подключение настройка новый оборудование пользователь прокладка диагностика поддержка офисный скс ведение заявка портал поддержка servicedesk требование опыт поддержка пользователь сервер терминал терминальный сервер удалённый рабочий стол отличный знание продукт семейство microsoft windows

In [ ]:
# график данных с hh.ru
sns.set(style="white")
corr = hh_dt.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 11, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio

sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot = True, vmin=-1, vmax=1, title = "Корреляция данных с hh.ru")
plt.show()

In [ ]:
# какой-то прикольный график
import plotly.express as px
plt.figure(figsize = (5,3.5))

fig = px.box(dt, x='schedule_name', y='salary_mean_net')
fig.show()

<Figure size 500x350 with 0 Axes>

In [ ]:
# графики зависимостей исходного датасета
plt.figure(figsize=(10,8), dpi= 80)
sns.pairplot(dt, kind="reg", hue="species")
plt.show()

In [ ]:
# графики зависимостей датасета по hh.ru
plt.figure(figsize=(10,8), dpi= 80)
sns.pairplot(hh_dt, kind="reg", hue="species")
plt.show()

## работа с данными данными

In [ ]:
X = dt.drop(['salary_mean_net', 'lemmaized_wo_stopwords_raw_description'], axis=1)
Y = dt.salary_mean_net



In [ ]:
import numpy as np 

X = np.concatenate( (X, vectors_transformed) , axis=1)
dt

,experience_name,schedule_name,unified_address_state,professional_roles_name,salary_mean_net,lemmaized_wo_stopwords_raw_description
0,1,0,0,0,80000.00,разрабатывать рекламный система оплата действи...
1,0,1,1,1,19989.12,обязанность ведение кадровый делопроизводство ...
2,1,1,0,2,45000.00,условие график работа 5/2 09.00 18.00;• оформл...
3,2,1,2,3,40000.00,строительный инженерный компания коммунальник ...
4,0,1,3,4,37500.00,обязанность знать устройство правило эксплуата...
...,...,...,...,...,...,...
61309,1,1,11,31,30450.00,обязанность контроль сварка вика контроль дета...
61310,0,1,62,20,30450.00,обязанность выполнять перечень работа услуга д...
61311,2,1,0,48,60000.00,обязанность ремонт реставрация одежда перекрой...
61312,0,4,27,7,13000.00,день сервис яндекс пользоваться миллион челове...


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


## Confusion matrix:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# from sklearn import svm, datasets
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
# from sklearn.utils.multiclass import unique_labels
# #X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)
# classifier = svm.SVC(kernel='linear', C=0.01)
# y_pred = classifier.fit(x_train, y_train).predict(X_test)
# #class_names = names

ValueError: Unknown label type: 'continuous'

In [ ]:
# def plot_confusion_matrix(y_true, y_pred, classes,
#                           normalize=False,
#                           title=None,
#                           cmap=plt.cm.Blues):

#     # Compute confusion matrix
#     cm = confusion_matrix(y_true, y_pred)
#     # Only use the labels that appear in the data
#     #classes = classes[unique_labels(y_true, y_pred)]
#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

#     fig, ax = plt.subplots()
#     im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
#     ax.figure.colorbar(im, ax=ax)
#     # We want to show all ticks...
#     ax.set(xticks=np.arange(cm.shape[1]),
#            yticks=np.arange(cm.shape[0]),
#            # ... and label them with the respective list entries
#            xticklabels=classes, yticklabels=classes,
#            title=title,
#            ylabel='True label',
#            xlabel='Predicted label')

#     # Rotate the tick labels and set their alignment.
#     #plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
#              #rotation_mode="anchor")

#     # Loop over data dimensions and create text annotations.
#     fmt = '.2f' if normalize else 'd'
#     thresh = cm.max() / 2.
#     for i in range(cm.shape[0]):
#         for j in range(cm.shape[1]):
#             ax.text(j, i, format(cm[i, j], fmt),
#                     ha="center", va="center",
#                     color="green")
#     fig.tight_layout()
#     return ax
#     return cm


# np.set_printoptions(precision=2)

# # Plot non-normalized confusion matrix
# #plot_confusion_matrix(y_test, y_pred, classes=classes,
#                       #title='Confusion matrix, without normalization')

# # Plot normalized confusion matrix
# plot_confusion_matrix(Y, preds, classes=classes, normalize=False,
#                       title='сonfusion matrix')

# plt.show()

### Метрики MAPE и MSE и MAE

In [ ]:
# import numpy as np

# def my_mape_score(y_true, y_pred):
#     ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
#     mape = 1/len(y_true) * np.sum(np.abs((y_pred - y_true) / y_true))

#     return mape

In [ ]:
# def my_mse_score(y_true, y_pred):
#     ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
#     mse = 1/len(y_true) * np.sum(np.square(y_pred - y_true))

#     return mse

In [ ]:
# def my_mae_score(y_true, y_pred):
#     ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
#     mae = 1/len(y_true) * np.sum(np.abs(y_pred - y_true))

#     return mae

Продолжение модельки:

In [ ]:
#model 1
from sklearn.linear_model import LinearRegression

modal = LinearRegression()
modal.fit(x_train, y_train)

LinearRegression()

In [ ]:
preds = modal.predict(x_test)

In [ ]:
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(y_test, preds))

15406.402937060535


In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, preds)
print(r2)

0.46399556490046423


In [ ]:
import numpy as np

def my_mape_score(y_test, preds):
    mape = 1/len(y_test) * np.sum(np.abs((preds - y_test) / y_test))

    return mape

my_mape_score(y_test, preds)

0.3508412023841485

In [ ]:
#model 2
from skl earn.tree import DecisionTreeRegressor
     
modal = DecisionTreeRegressor(max_depth=4, random_state=17, criterion="absolute_error")
modal.fit(x_train, y_train)
preds = modal.predict(x_test)

In [ ]:
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(y_test, preds))

16968.170696686826


In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, preds)
print(r2)

0.26589944107894525


In [ ]:
import numpy as np

def my_mape_score(y_test, preds):
    mape = 1/len(y_test) * np.sum(np.abs((preds - y_test) / y_test))

    return mape

my_mape_score(y_test, preds)

0.3466514550701091

In [ ]:
#model 3
from sklearn.ensemble import RandomForestRegressor

modal = RandomForestRegressor(max_depth=4, random_state=0)
modal.fit(x_train, y_train)
preds = modal.predict(x_test)


In [ ]:
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(y_test, preds))

17367.94372008865


In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, preds)
print(r2)

0.34167702503074815


In [ ]:
import numpy as np

def my_mape_score(y_test, preds):
    mape = 1/len(y_test) * np.sum(np.abs((preds - y_test) / y_test))

    return mape

my_mape_score(y_test, preds)

0.4102082785691639

### Объединение данных:

In [ ]:
data_add_main = data_add[['experience', 'schedule', 'area', 'professional_roles', 'salary']]
data_add_main = data_add_main.rename(columns={'experience': 'experience_name', 'schedule': 'schedule_name', 'area': 'unified_address_state', 'professional_roles': 'professional_roles_name', 'salary': 'salary_mean_net'})
data_add_main

#hh_data = pd.concat([dt, data_add_main], axis=0)
hh_data = data_add_main

In [ ]:
import pandas as pd
pd.read_csv('big_data.csv').shape

(62599, 6)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5b2b2bc4-db2b-4888-b30d-5a6b30dbe27f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>